In [5]:
import torch
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F

In [6]:
import pandas as pd
conveyors = pd.read_csv("data/Conveyors.csv", quotechar='"')
conveyors.head()

,Position,Name,Quantity,ConveyorWidth_mm,BeltWidth_mm,DriveToTensionDistance_m,BeltLength_m,NetBeltLength_m,Facility Area,Condition
0,1,40,1,2000,1600,196,407,398,1,"Работает с 1988г., металическая часть подвергл..."
1,2,41,1,2000,1600,202,427,408,1,NaN
2,3,42,1,2000,1600,32,70,67,1,NaN
3,4,41a,1,2000,1600,17,39,36,1,Работает с 2003г. металическая часть подвергла...
4,5,42a,1,1200,1600,28,61,58,1,Работает с 2019г. В нормальном состаяние.Бетон...


In [7]:
def bidirectional_chain_edges(n):
    forward = torch.stack([torch.arange(n-1), torch.arange(1, n)])
    backward = torch.stack([torch.arange(1, n), torch.arange(n-1)])
    edges = torch.cat([forward, backward], dim=1)
    return edges

In [8]:
x = torch.tensor(
    conveyors.loc[conveyors["Name"]
                    .isin(['45', '46', '47'])]
                    [['BeltWidth_mm', 'BeltLength_m']].values, 
                  dtype=torch.float32)

edges = bidirectional_chain_edges(3)

data = Data(x=x, edge_index=edges)
print(data)

class ConveyorGCN(nn.Module):
    def __init__(self, in_feats, hidden_feats, out_feats):
        super().__init__()
        self.conv1 = GCNConv(in_feats, hidden_feats)
        self.conv2 = GCNConv(hidden_feats, out_feats)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = F.relu(h)
        h = self.conv2(h, edge_index)
        return h

# Ենթադրենք, որ ցանկանում ենք կանխատեսել 2 ելք յուրաքանչյուր հանգույցի համար,
# օրինակ՝ [գերտաքացման հավանականություն, որոշակի լրացուցիչ չափորոշիչ]
model = ConveyorGCN(in_feats=2, hidden_feats=4, out_feats=2)

# Ուղիղ անցում (առայժմ առանց ուսուցման, կշիռները պատահական են)
out = model(data.x, data.edge_index)
print("Մոդելի հում ելքեր (լոգիտներ):\n", out)

# Փոխարկում ենք դասերի հավանականությունների (եթե դասակարգման խնդիր է, օրինակ՝ 2 դաս)
probs = F.softmax(out, dim=1)
print("Հավանականություններ (softmax-ից հետո):\n", probs)


Data(x=[3, 2], edge_index=[2, 4])
Մոդելի հում ելքեր (լոգիտներ):
 tensor([[-524.9824,  101.2301],
        [-645.0829,  122.1134],
        [-532.5560,   98.8153]], grad_fn=<AddBackward0>)
Հավանականություններ (softmax-ից հետո):
 tensor([[0., 1.],
        [0., 1.],
        [0., 1.]], grad_fn=<SoftmaxBackward0>)
